In [ ]:
import matplotlib.pyplot as plt
from scipy import interpolate
import numpy as np
import pickle
import pandas as pd
from IPython.display import clear_output
import time

import pyomo.environ as pyo
import pyomo.dae as pde
import tclab

from ORCA.Optimization.DAE_LTIStateSpaceMPCPyomoOptimization import (
    DAE_LTIStateSpaceMPCPyomoOptimization,
)

## Load and test surrogate models

load pretrained models that accurately represents the dynamics of system transients

In [ ]:
filename = "ORCA/tclab_data2.csv"
data = pd.read_csv(filename)

t_sim = data["time"].values
q1_sim = data["Q1"].values
q2_sim = data["Q2"].values
t1_setpoint = data["T1"].values
t2 = data["T2"].values

tclab_setpoint = interpolate.interp1d(t_sim, t1_setpoint)

plt.figure(figsize=(10, 8))
plt.subplot(3, 1, 1)
plt.plot(t_sim, t1_setpoint)
plt.title("Setpoint")
plt.ylabel("deg C")
plt.grid(True)

plt.subplot(3, 1, 2)
plt.plot(t_sim, q1_sim)
plt.title("Heater 1 power input")
plt.ylabel("percent of max")
plt.grid(True)

plt.subplot(3, 1, 3)
plt.plot(t_sim, q2_sim)
plt.title("Heater 2 power input")
plt.ylabel("watts")
plt.grid(True)

plt.xlabel("Time (s)")

plt.tight_layout()

In [ ]:
matrices_path = "ORCA/ABC.pkl"
with open(matrices_path, "rb") as f:
    matrices = pickle.load(f)

# store matrices
A = matrices["A"]
B = matrices["B"]
C = matrices["C"]

In [ ]:
opt_specs = {
    "q1_sim": data["Q1"].values,
    "q2_sim": data["Q2"].values,
    "t1_setpoint": data["T1"].values,
    "t": None,
    "t_window": 600.0,
    "dt": 1.0,
    "matrices": "ORCA/ABC.pkl",
    "states": {
        "order": ["T1s", "T2s", "T1sp", "T2sp"],
        "lb": [0.0, 0.0, 0.0, 0.0],
        "ub": [300.0, 300.0, 300.0, 300.0],
    },
    "control": {"order": ["Q1", "Q2"], "lb": [0.0, 0.0], "ub": [100.0, 100.0]},
    "measurements": {"order": ["T1m", "T2m"], "lb": [0.0, 0.0], "ub": [300.0, 300.0]},
    # objective is not used, just dummy
    "objective": {
        "sense": "minimize",
    },
}
optimize = DAE_LTIStateSpaceMPCPyomoOptimization(solver="ipopt", mode=0, **opt_specs)
print(optimize.model.y)

In [ ]:
T1_sim = np.array([optimize.model.y[0, t]() for t in t_sim])
T2_sim = np.array([optimize.model.y[1, t]() for t in t_sim])

# visualization
plt.subplot(2, 1, 1)
plt.plot(t_sim, T1_sim + t1_setpoint[0], "r--", label="simulated")
# plt.plot(t_sim, T2_sim+t2[0])
plt.plot(t_sim, t1_setpoint, color="red", label="actual")
plt.title("temperatures")
plt.ylabel("deg C")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(t_sim, np.array([optimize.model.u[0, t]() for t in t_sim]), "r")
plt.plot(t_sim, np.array([optimize.model.u[1, t]() for t in t_sim]), "b")
plt.title("heater power")
plt.ylabel("percent of max")
plt.grid(True)

plt.tight_layout()

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)

    greater_indices = np.where(array >= value)[0]

    # Find the index of the nearest element
    nearest_index = greater_indices[np.argmin(array[greater_indices] - value)]
    return nearest_index

In [ ]:
def tclab_control(t, h, x_init=[0, 0, 0, 0], u_init=[0, 0]):
    tf = t + h
    opt_specs2 = {
        "t_sim": np.linspace(t, tf, 1 + round(tf - t)),
        "q1_sim": data["Q1"].values,
        "q2_sim": data["Q2"].values,
        "t1_setpoint": tclab_setpoint,
        "t": t,
        "solver": "ipopt",
        "matrices": "ORCA/ABC.pkl",
        "states": {
            "order": ["T1s", "T2s", "T1sp", "T2sp"],
            "lb": [0.0, 0.0, 0.0, 0.0],
            "ub": [300.0, 300.0, 300.0, 300.0],
        },
        "control": {"order": ["Q1", "Q2"], "lb": [0.0, 0.0], "ub": [100.0, 100.0]},
        "measurements": {
            "order": ["T1m", "T2m"],
            "lb": [0.0, 0.0],
            "ub": [300.0, 300.0],
        },
        "objective": {
            "sense": "minimize",
        },
    }
    optimize = DAE_LTIStateSpaceMPCPyomoOptimization(mode=1, **opt_specs2)

    tsim = np.array([t for t in optimize.model.t])
    T1sim = np.array([optimize.model.y[0, t]() + x_init[2] for t in optimize.model.t])
    T2sim = np.array([optimize.model.y[1, t]() + x_init[3] for t in optimize.model.t])
    Q1sim = np.array([optimize.model.u[0, t]() for t in optimize.model.t])
    Q2sim = np.array([optimize.model.u[1, t]() for t in optimize.model.t])
    T1sp = np.array([tclab_setpoint(t) for t in optimize.model.t])
    return [tsim, Q1sim, Q2sim, T1sim, T2sim, T1sp]

In [ ]:
dt = 2  # in seconds

a = tclab.TCLabModel()

tclab_setpoint = interpolate.interp1d(
    [0, 50, 150, 450, 550, 9999], [21, 21, 60, 60, 35, 35]
)

x_init = [0, 0, a.T1, a.T2]

# specify an initial and end time
initial_time = pd.to_datetime("2022-05-31 12:00:00")

start_time = pd.to_datetime("2022-05-31 12:00:00")
end_time = pd.to_datetime("2022-05-31 12:10:00")

prd_time = start_time
current_time = start_time
prev_time = time.time()

result_df = None

while current_time < end_time:
    print(current_time)

    t = (current_time - initial_time).total_seconds()
    h = 5 * dt

    [tsim, Q1sim, Q2sim, T1sim, T2sim, T1sp] = tclab_control(t, h, x_init)

    idx_time = find_nearest(tsim, t + dt)
    print(tsim)
    print(t + dt)
    print(
        "the nearest time index equal to dt is {} at {}".format(
            idx_time, tsim[idx_time]
        )
    )

    # Sleep time
    sleep_max = 2.0
    sleep = sleep_max - (time.time() - prev_time)
    if sleep >= 0.01:
        time.sleep(sleep)
    else:
        print("Warning: cycle time too fast")
        print("Requested: " + str(sleep_max))
        print("Actual: " + str(time.time() - prev_time))
        time.sleep(0.01)
    prev_time = time.time()

    a.Q1(Q1sim[idx_time])
    a.Q2(Q2sim[idx_time])

    optimal_dict = {"Time": [current_time]}
    optimal_dict["T1m"] = a.T1
    optimal_dict["T2m"] = a.T2
    optimal_dict["Q1s"] = Q1sim[idx_time]  # 2second intervals
    optimal_dict["Q2s"] = Q2sim[idx_time]
    optimal_dict["T1s"] = T1sim[idx_time]
    optimal_dict["T2s"] = T2sim[idx_time]
    optimal_dict["T1sp"] = T1sp[idx_time]

    optimal = pd.DataFrame(optimal_dict)
    if result_df is None:
        result_df = optimal
    else:
        result_df = pd.concat([result_df, optimal], ignore_index=True)

    print(optimal)

    x_init = [0, 0, optimal_dict["T1m"], optimal_dict["T2m"]]

    time.sleep(1)
    current_time += pd.Timedelta(seconds=dt)

a.Q1(0)
a.Q2(0)
a.close()

In [ ]:
sim_time = (result_df["Time"] - initial_time).dt.total_seconds()
Tset = [tclab_setpoint(t) for t in sim_time]

In [ ]:
plt.subplot(2, 1, 1)
plt.plot(sim_time, result_df["T1m"].values, color="red", label="measured value")
plt.plot(sim_time, result_df["T1s"].values, color="blue", label="simulated value")
plt.plot(sim_time, Tset, "k--", label="setpoint")
plt.title("temperatures")
plt.ylabel("deg C")
plt.legend()
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(sim_time, result_df["Q1s"], color="red", label="heater 1")
plt.plot(sim_time, result_df["Q2s"], color="blue", label="heater 2")
plt.title("heater input")
plt.ylabel("percent of max")
plt.grid(True)

plt.tight_layout()